In [1]:
import time
from detector import FaceDetector
import cv2
import av
from IPython.display import Video

In [2]:
# Функция для отображения FPS на кадре с измененным положением текста
def draw_fps(frame, fps):
    font_scale = 2.5
    font_thickness = 4
    text = f"FPS: {int(fps)}"
    # Получение размеров текста
    (text_width, text_height), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness)
    # Положение текста в левом верхнем углу
    position = (10, text_height + 10)  # Смещаем текст вниз на высоту текста плюс некоторый отступ
    cv2.putText(frame, text, position, cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 0, 0), font_thickness)

In [4]:
face_detector = FaceDetector(model_path='best.pt',
                             device='cpu',
#                              crops_path="crops/",
#                              meta_path="meta/meta.json",
#                              marks_path="marks/marks.json"
                            )

input_filename = "assets/train_station.mp4"
output_filename = "inference_train_station.mp4"

input_container = av.open(input_filename)
output_container = av.open(output_filename, "w")
stream = output_container.add_stream("h264")

frame_count = 0
fps = 0
# Запись изображений в видео
for frame in input_container.decode(video=0):
    start_time = time.time()
    if frame_count > 500:
        break
    # Получаем изображение кадра
    image = frame.to_ndarray(format='rgb24')

    # Обрабатываем кадр с помощью детектора лиц
    results = face_detector.update(image)
    
    fps = 1 / (time.time() - start_time)
    
    annotated_frame = results[0].plot()
    draw_fps(annotated_frame, fps)
    
    frame = av.VideoFrame.from_ndarray(annotated_frame, format="rgb24")
    packet = stream.encode(frame)
    output_container.mux(packet)
    
    print("fps: ", fps)
    frame_count += 1
        
# Завершение записи видео
packet = stream.encode(None)  # Посылаем None, чтобы завершить видеопоток
output_container.mux(packet)
# Закрытие видеопотока
output_container.close()
input_container.close()

print(f"Видео сгенерировано: {output_filename}")

Ultralytics YOLOv8.1.27 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-32GB, 32501MiB)
Setup complete ✅ (80 CPUs, 754.5 GB RAM, 4130.3/4434.8 GB disk)

0: 384x640 4 faces, 86.2ms
Speed: 2.7ms preprocess, 86.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
fps:  0.6696645581372477

0: 384x640 4 faces, 33.0ms
Speed: 2.1ms preprocess, 33.0ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
fps:  22.578548165693213

0: 384x640 4 faces, 31.3ms
Speed: 2.0ms preprocess, 31.3ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
fps:  23.73659607701101

0: 384x640 4 faces, 30.6ms
Speed: 1.8ms preprocess, 30.6ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
fps:  25.692205914782054

0: 384x640 4 faces, 32.0ms
Speed: 1.8ms preprocess, 32.0ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
fps:  24.049632458343368

0: 384x640 4 faces, 28.2ms
Speed: 1.9ms preprocess, 28.2ms inference, 0.7ms postprocess

In [17]:
Video("inference_train_station.mp4", width=990, height=360)